In [2]:
import numpy as np
import math

import sys, os
sys.path.append("../")

from iotools.readOxford import ReadOxford
from iotools.readrpkm import ReadRPKM
from iotools.io_model import WriteModel
from inference.linreg_association import LinRegAssociation
from inference.empirical_bayes import EmpiricalBayes
from utils import hyperparameters
from inference import logmarglik
from iotools import readgtf
from utils import gtutils
from utils import mfunc
from utils.containers import ZstateInfo
from utils.printstamp import printStamp

from sklearn.preprocessing import scale

home = '/home/fsimone'
gtfpath = os.path.join(home,"datasets/gtex/gencode.v19.annotation.gtf.gz")
chrom = 1
samplepath = os.path.join(home, "datasets/gtex/donor_ids.fam")
gtpath = os.path.join(home, "datasets/gtex/GTEx_450Indiv_genot_imput_info04_maf01_HWEp1E6_dbSNP135IDs_donorIDs_dosage_chr"+str(chrom)+".gz")
rpkmpath = os.path.join(home, "datasets/gtex/gtex_wholeblood_normalized.expression.txt")
params = [0.1, 0.0, 0.01, 0.001, 0.005]
zmax = 1
# for testing purposes I keep using the previous gene expression
# rpkmpath = os.path.join(home, "datasets/gtex/gtex_wholeblood_normalized.lm_corr_final2.exp.txt"


In [4]:
# Annotation (use complete gene name in gtf without trimming the version)
gene_info = readgtf.gencode_v12(gtfpath, include_chrom = chrom, trim=False)

In [5]:
# read Genotype
oxf = ReadOxford(gtpath, samplepath, chrom, "gtex")
genotype = np.array(oxf.dosage)
samplenames = oxf.samplenames
snps = oxf.snps_info

2018-02-27 17:19:35 - started reading genotype
Read 892083 snps in 450 samples.
2018-02-27 17:21:31 - Finished readings snps


In [6]:
# Quality control
f_snps, f_genotype = gtutils.remove_low_maf(snps, genotype, 0.1)
gt = gtutils.normalize(f_snps, f_genotype)


In [7]:
# Gene Expression
rpkm = ReadRPKM(rpkmpath, "gtex")
expression = rpkm.expression
expr_donors = rpkm.donor_ids
gene_names = rpkm.gene_names


In [8]:
# Selection
printStamp("Selection of samples")
vcfmask, exprmask = mfunc.select_donors(samplenames, expr_donors)
genes, indices = mfunc.select_genes(gene_info, gene_names)


min_snps = 200
pval_cutoff = 0.001
window = 1000000
zmax = zmax    # z parameter
init_params = np.array(params)
init_params[4] = 1 / init_params[4] / init_params[4]

model = WriteModel(".", chrom)

batch_size = None
gene_number = len(genes)

2018-02-27 17:22:55 - Selection of samples


In [15]:
# load gene ids of interest

gene_list = []
# genelistfile = "target_gene_list" # r2 > 0.1 and px > 2*gx
genelistfile = "genes4testing_highr2" # r2 > 0.1
r2val = []
predixcan_r2val = []
with open(genelistfile, 'r') as instream:
    for line in instream:
        arr = line.strip().split()
        gene_list.append(arr[0])
        r2val.append(arr[1])
        predixcan_r2val.append(arr[2])

print("Read {:d} high r2 genes\n".format(len(gene_list)))
        
selected_gene_ids = []
for i in gene_info:
    trimid = i.ensembl_id.split(".")[0]
    if trimid in gene_list:
        print("Gene {:s}, CHR {:d}, R2 value: {:s} ".format(i.ensembl_id, i.chrom, r2val[gene_list.index(trimid)]))
        # print("R2 value: ",r2val[gene_list.index(trimid)], "\t", predixcan_r2val[gene_list.index(trimid)])
        selected_gene_ids.append(i.ensembl_id)

Read 70 high r2 genes

Gene ENSG00000158825.5, CHR 1, R2 value: 0.3754060956719503 
Gene ENSG00000183386.5, CHR 1, R2 value: 0.4036021829816217 
Gene ENSG00000116157.5, CHR 1, R2 value: 0.4385918707779852 


In [10]:
for i, gene in enumerate(genes):
    k = indices[i]
    if gene.ensembl_id in selected_gene_ids[0]:
        break
print(k,gene)

342 GeneInfo(name='CDA', ensembl_id='ENSG00000158825.5', chrom=1, start=20915440, end=20945401)


In [11]:

# select only the cis-SNPs
cismask = mfunc.select_snps(gene, f_snps, window)
if len(cismask) > 0:
    target = expression[k, exprmask]
    target = scale(target, with_mean=True, with_std=True)
    predictor = gt[cismask][:, vcfmask]
    snpmask = cismask

    # if number of cis SNPs > threshold, use p-value cut-off
    if len(cismask) > min_snps:
        assoc_model = LinRegAssociation(predictor, target, min_snps, pval_cutoff)
        pvalmask = cismask[assoc_model.selected_variables]
        print ("Found {:d} SNPs, reduced to {:d} SNPs (max p-value {:g}) for {:s}".format(len(cismask), len(pvalmask), assoc_model.ordered_pvals[len(pvalmask) - 1], gene.name))
        predictor = gt[pvalmask][:, vcfmask]
        snpmask = pvalmask
    else:
        print ("Found {:d} SNPs for {:s}".format(len(cismask), gene.name))

Found 3096 SNPs, reduced to 200 SNPs (max p-value 0.0441457) for CDA


In [ ]:
# perform the analysis

print ("Starting first optimization ==============")
emp_bayes = EmpiricalBayes(predictor, target, 1, init_params, method="new")
emp_bayes.fit()
if zmax > 1:
    if emp_bayes.success:
        res = emp_bayes.params
        print ("Starting second optimization from previous results ================")
        # Python Error: C library could not compute z-components. Check C errors above.
    else:
        res = init_params
        print ("Starting second optimization from initial parameters ================")
    emp_bayes = EmpiricalBayes(predictor, target, zmax, res, method="new")
    emp_bayes.fit()

Starting first optimization ==============
1.385140400408024e-45 0.98
Working with 2 zstates.
0.00014247987205086657 0.98
Working with 24 zstates.
0.0025906558395850546 0.98
Working with 135 zstates.
0.024697271289687944 0.98
Working with 194 zstates.
0.03508719074260826 0.98
Working with 196 zstates.
0.03973851447855654 0.98
Working with 197 zstates.
0.04151498115030569 0.98
Working with 197 zstates.
0.042331079844077094 0.98
Working with 197 zstates.
0.04274874637125797 0.98
Working with 197 zstates.
0.043005388881112196 0.98
Working with 197 zstates.
0.043223925053593404 0.98
Working with 197 zstates.
0.04350822247904199 0.98
Working with 197 zstates.
0.0439728928144358 0.98
Working with 197 zstates.


In [116]:
if emp_bayes.success:
    res = emp_bayes.params
    res[4] = 1 / np.sqrt(res[4])
    print("PI: \t",res[0])
    print("mu: \t",res[1])
    print("sigma: \t",res[2])
    print("sigmabg: \t",res[3])
    print("tau: \t",res[4])

    model_snps = [snps[x] for x in snpmask]
    model_zstates = list()
    scaledparams = hyperparameters.scale(emp_bayes.params)
    zprob, zexp = logmarglik.model_exp(scaledparams, predictor, target, emp_bayes.zstates)
    for j, z in enumerate(emp_bayes.zstates):
        this_zstate = ZstateInfo(state = z,
                                 prob  = zprob[j],
                                 exp   = list(zexp[j, :]) )
        model_zstates.append(this_zstate)
    # print(model_snps)
    # for i,m in enumerate(model_zstates):
    #     print("z-state: ",i," Prob:", m.prob)
    model.write_success_gene(gene, model_snps, model_zstates, res)
else:
    model.write_failed_gene(gene, np.zeros_like(init_params))
    print ("Failed optimization")

PI: 	 0.000226820116643532
mu: 	 0.0
sigma: 	 0.001000044291115713
sigmabg: 	 0.03400516423944316
tau: 	 0.8749300969843092
z-state:  0  Prob: 0.9760830572660332
z-state:  1  Prob: 0.00022154478879425203
z-state:  2  Prob: 0.0002214549039606261
z-state:  3  Prob: 0.00022145491104533325
z-state:  4  Prob: 0.00022145469584534133
z-state:  5  Prob: 0.0002214546912404114
z-state:  6  Prob: 0.00022145489721834484
z-state:  7  Prob: 0.00022145099397045086
z-state:  8  Prob: 0.00022145127034862408
z-state:  9  Prob: 0.00022147731630414188
z-state:  10  Prob: 0.00022146784510024799
z-state:  11  Prob: 0.00022146808110090995
z-state:  12  Prob: 0.00022144431983903975
z-state:  13  Prob: 0.00022147789257922392
z-state:  14  Prob: 0.00022144289727708054
z-state:  15  Prob: 0.00022144279585390556
z-state:  16  Prob: 0.00022144669761806142
z-state:  17  Prob: 0.00022144243913084128
z-state:  18  Prob: 0.00022145422694530245
z-state:  19  Prob: 0.0002214550311682283
z-state:  20  Prob: 0.00022145435

In [118]:
print("PI: \t",res[0])
print("mu: \t",res[1])
print("sigma: \t",res[2])
print("sigmabg: \t",res[3])
print("tau: \t",res[4])


PI: 	 0.000226820116643532
mu: 	 0.0
sigma: 	 0.001000044291115713
sigmabg: 	 0.03400516423944316
tau: 	 0.8749300969843092


# Prediction

In [108]:
modelpath="."
cardio_home = os.path.join(home,"datasets/cardiogenics/")
p_gtpath=os.path.join(cardio_home, "genotypes/CG_"+str(chrom)+".imputed.gz")
p_samplepath=os.path.join(cardio_home, "genotypes/CG.sample")
dataset="cardiogenics"

pred_path=os.path.join(home,"gxpred/devtools")
outfileprefix = os.path.join(pred_path,"pred_chr"+str(chrom))

# Actual Prediction

In [109]:
# Model
from iotools.io_model import ReadModel
from utils.containers import GeneExpressionArray
from utils import mfunc

p_model = ReadModel(modelpath, chrom)
genes = p_model.genes
gx = list()
for gene in genes:

    p_model.read_gene(gene)
    p_model_snps = p_model.snps
    p_model_zstates = p_model.zstates

    x = gtutils.prediction_variables(p_snps, p_model_snps, p_genotype)
    x = gtutils.normalize(p_model_snps, x)

    ypred = np.zeros(p_nsample)
    for z in p_model_zstates:
        ypred += z.prob * np.dot(x.T, z.exp)

    gx.append(GeneExpressionArray(geneid = gene.ensembl_id, expr_arr = ypred))


# Write output
mfunc.write_gcta_phenotype(outfileprefix, samplenames, gx)
# printStamp("End Prediction")

# Comparison to measured expression data

In [96]:
# read Cardiogenics Genotype

p_oxf = ReadOxford(p_gtpath, p_samplepath, chrom, dataset)
p_genotype = np.array(p_oxf.dosage)
p_samplenames = p_oxf.samplenames
p_snps = p_oxf.snps_info
p_nsample = len(p_oxf.samplenames)

2018-02-22 18:42:43 - started reading genotype
Read 1260727 snps in 786 samples.
2018-02-22 18:55:32 - Finished readings snps


In [103]:
# Read Cardiogenics expression data

from iotools.readrpkm import ReadRPKM
expdatapath = os.path.join(pred_home, "expression/lumi_mono_all_vsn_batches_adjusted.RData.exp.txt")

# Load dataset Gene Expression
rpkm = ReadRPKM(expdatapath, "cardiogenics")
expression = rpkm.expression
expr_donors = rpkm.donor_ids
gene_names = rpkm.gene_names

In [46]:
import re

class ReadPrediction:
    
    
    def __init__(self, predpath, samplepath, predictor):
        self._pred_path = predpath
        self._predictor = predictor #gxpred or predixcan
        self._expr_mat = None
        self._samplepath = samplepath
        self._load_samples()
        self._load_chromosomes()
    
    def _load_samples(self):
        samples = list()
        with open(self._samplepath, 'r') as instream:
            for line in instream:
                if re.search('^#', line):
                    continue
                arr = line.strip().split()
                samples.append(arr[0])
        self._samples = samples            
    
    # return expression matrix in samples-by-genes format
    def _load_chromosome(self, chrom):
        
        if self._predictor == "gxpred":
            basefile = "pred_chr"+chrom
            pred_geneids = os.path.join(self._pred_path,basefile+".geneids")
            pred_exp = os.path.join(self._pred_path,basefile+".txt")
            
            with open(pred_geneids, 'r') as infile:
                gene_names = [l.strip() for l in infile.readlines()]
                ncolumns = len(gene_names)
            
            # need the number of columns (num of genes), to skip first two sample id columns
            expr_mat = np.loadtxt(pred_exp, usecols=range(2,ncolumns+2))            
            
        if self._predictor == "predixcan":
            filename = "predixcan_chr"+chrom+".output"
            filepath = os.path.join(self._pred_path, filename)
            with open(filepath, 'r') as instream:
                gene_names = instream.readline().strip().split()
            expr_mat = np.loadtxt(filepath, skiprows=1)
            
        print(expr_mat.shape)
        print("genes: ",len(gene_names))
        return expr_mat, gene_names
                
    def _load_chromosomes(self):
        self._gene_names = []
        self._expr_mat = None
               
        for chrom in range(1,23):
            expr_mat, gene_names = self._load_chromosome(str(chrom))
            if self._predictor == "gxpred":
                if type(self._expr_mat) is not np.ndarray:
                    self._expr_mat = expr_mat.T
                else:
                    self._expr_mat = np.concatenate((self._expr_mat, expr_mat.T), axis=0)
                self._gene_names += gene_names
            
            if self._predictor == "predixcan":
                if type(self._expr_mat) is not np.ndarray:
                    self._expr_mat = expr_mat
                else:
                    self._expr_mat += expr_mat
                self._gene_names = gene_names

    def sort_by_samples(self, samplelist, use_prev = False):
        # sorts expression matrix by the samplelist
        # writes to sorted_expr_mat
        # if use_prev == True, should use previously sorted matrix, samples and gene
        # else: use original expr_mat, genes and samples
        
        # sort sample names for px gx and cardiogenics

        # since they are the same, the index will be same for both (pgx)

        common_samples = [x for x in expr_donors if x in gx_samples]
        rgx_samples_index = [gx_samples.index(i) for i in common_samples]
        cardio_samples_index = [expr_donors.index(i) for i in common_samples]

        rgx_sorted_samples = [gx_samples[i] for i in rgx_samples_index]
        cardio_sorted_samples = [expr_donors[i] for i in cardio_samples_index]

        # are Samples indexes ok?
        print(rgx_sorted_samples == cardio_sorted_samples)
        print(len(rgx_sorted_samples))
        print(len(cardio_sorted_samples))
        pass
    
    def sort_by_gene(self, genelist, use_prev = False):
        # sorts expression matrix by the given genelist
        
        # sort gene names
        print("Total cardio genes:",len(gene_names))

        gx_gene_names2 = [x.split(".")[0] for x in gx_gene_batch]
        rand_gx_gene_names2 = [x.split(".")[0] for x in rand_gx_gene_batch]

        # I don't know WHY random gxpred has less genes!! intersection should be 11642 and not 11179
        #set_a = set(gx_gene_batch)
        #set_b = set(rand_gx_gene_batch)
        #set_c = set_a.intersection(set_b)
        #print(len(set_a), len(set_b), len(set_c))

        set_rgx = set(rand_gx_gene_names2)
        set_gx = set(gx_gene_names2)

        common_genes = set_rgx.intersection(set_gx)
        print("rgx and gx common genes: ",len(common_genes))

        cardio_common_genes = [g for g in common_genes if g in gene_names]

        print("cardio,rgx,gx, common genes: ", len(cardio_common_genes))

        # this is for indexing cardiogenics expression matrix
        gx_cardio_gene_index = [gene_names.index(i) for i in cardio_common_genes]
        print(len(gx_cardio_gene_index),"/",len(gene_names))

        # this is for indexing gx and px gene names and expression mat
        gx_gene_index = [gx_gene_names2.index(i) for i in cardio_common_genes]
        rgx_gene_index = [rand_gx_gene_names2.index(i) for i in cardio_common_genes]

        print(len(gx_gene_index),"/",len(gx_gene_batch))
        print(len(rgx_gene_index),"/",len(rand_gx_gene_batch))
        pass
    
    
    
    @property
    def expr_mat(self):
        return self._expr_mat
    
    @property
    def gene_names(self):
        return self._gene_names
    
    @property
    def samples(self):
        return self._samples

In [47]:
gxpred_predpath = "/home/franco/cbscratch/datasets/cardiogenics/gxpred_predictions_klinikum"
samplepath = "/home/franco/cluster2/datasets/cardiogenics/genotypes/CG.sample"
gxpred = ReadPrediction(gxpred_predpath, samplepath, "gxpred")

(786, 1554)
genes:  1554
(786, 952)
genes:  952
(786, 835)
genes:  835
(786, 523)
genes:  523
(786, 639)
genes:  639
(786, 709)
genes:  709
(786, 614)
genes:  614
(786, 502)
genes:  502
(786, 592)
genes:  592
(786, 556)
genes:  556
(786, 824)
genes:  824
(786, 830)
genes:  830
(786, 245)
genes:  245
(786, 487)
genes:  487
(786, 445)
genes:  445
(786, 672)
genes:  672
(786, 864)
genes:  864
(786, 203)
genes:  203
(786, 1160)
genes:  1160
(786, 395)
genes:  395
(786, 148)
genes:  148
(786, 375)
genes:  375


In [48]:
gxpred.expr_mat.shape

(14124, 786)

In [49]:
pxpred_predpath = "/home/franco/cbscratch/datasets/cardiogenics/predixcan_predictions_klinikum"
samplepath = "/home/franco/cluster2/datasets/cardiogenics/genotypes/CG.sample"
predixcanpred = ReadPrediction(pxpred_predpath, samplepath, "predixcan")

(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759
(786, 6759)
genes:  6759


In [50]:
predixcanpred.expr_mat.shape

(786, 6759)

In [101]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(, gx.expr_arr)
ax.plot([0,1], [0,1])
plt.show()

[GeneExpressionArray(geneid='ENSG00000183386.5', expr_arr=array([ 5.76509206e-02,  2.30666594e-01, -1.03668846e-01, -3.38784261e-01,
        1.68540142e-01, -2.97173274e-03, -2.17672991e-01, -1.81710250e-01,
       -1.38896980e-01, -1.05242816e-01,  6.79431197e-02, -1.29499674e-01,
       -2.76376253e-01, -1.44089024e-01,  1.64167333e-01, -3.34455286e-01,
        1.33228357e-01, -2.80445636e-01, -2.09993032e-01, -2.17784174e-01,
       -4.25467932e-02,  7.11238411e-01,  5.99397386e-03, -5.13565772e-02,
       -3.27922275e-01,  1.67855406e-01, -2.41047709e-01,  2.72924269e-01,
        9.14703384e-02, -2.73657620e-01,  2.63657323e-01, -4.24768146e-01,
        3.64469610e-02,  8.68841089e-02, -3.00838751e-02, -1.35982421e-01,
        3.70550094e-03, -2.03859379e-01, -2.19058561e-01,  2.09547526e-01,
       -8.56014635e-02,  2.67564629e-01, -1.54836684e-01,  6.21477823e-02,
        2.48127600e-01,  2.45707722e-02,  2.08148523e-01, -2.28517779e-01,
       -1.85447030e-01,  3.77538157e-01, -